#### Converting from world to pixel + creating a map file for each nodule (counting by nodules and not by exam)

In [1]:
import numpy as np
import json
from utils import (
    load_itk, world_to_pixel
)
from tqdm import tqdm

In [2]:
files = [
    'parsed-mixed_test-overjoyed-hen-305-standard',
    'parsed-mixed_test-indecisive-eel-172-standard',
    'parsed-mixed_test-crawling-sloth-537-standard',
    'parsed-mixed_test-powerful-goat-766-standard'
]

In [3]:
for file_name in files:
    ### loading file
    data_list_file_path = f'/data/output/validation/same_window/{file_name}.json'
    f = open(data_list_file_path)
    data = json.load(f)

    ### perform convertion
    converted = []
    for content in tqdm(data['test']):

        img, origin, spacing = load_itk(content['image'])
        origin = np.array(list(reversed(origin)))
        spacing = np.array(list(reversed(spacing)))

        for true_box in content['true_box']:

            targets = []

            world_coord_cent = np.array((true_box[0], true_box[1], true_box[2]))
            world_coord_diam = np.array((true_box[3], true_box[4], true_box[5]))

            voxel_min, voxel_max = world_to_pixel(
                origin, spacing, world_coord_cent, world_coord_diam
            )

            targets.append({
                "voxel_min": voxel_min.astype(int).tolist(),
                "voxel_max": voxel_max.astype(int).tolist()
            })

            boxes = []
            scores = []

            for pred_box, score in zip(content['pred_box'], content['score']):

                # if score > 0.5:
                world_coord_cent = np.array((pred_box[0], pred_box[1], pred_box[2]))
                world_coord_diam = np.array((pred_box[3], pred_box[4], pred_box[5]))

                voxel_min, voxel_max = world_to_pixel(
                    origin, spacing, world_coord_cent, world_coord_diam
                )

                boxes.append({
                    "voxel_min": voxel_min.astype(int).tolist(),
                    "voxel_max": voxel_max.astype(int).tolist()
                })

                scores.append(score)
        
            converted.append({
                "image": content['image'],
                "true_box": targets,
                "pred_box": boxes,
                "score": scores
            })

    ### saving converted data
    value = {
        "test": converted
    }
    with open(f'/data/output/validation/same_window/pixel-{file_name}.json', 'w') as write_file:
        json.dump(value, write_file, indent=4)

100%|██████████| 155/155 [05:30<00:00,  2.13s/it]
